In [279]:
#read file

def ReadFile(data_set_number):
    
    if data_set_number==0:
        file_path = 'a280-n1395.txt'
    elif data_set_number==1:    
        file_path = 'a280-n279.txt'
    elif data_set_number==2: 
        file_path = 'a280-n2790.txt'
    elif data_set_number==3: 
        file_path = 'fnl4461-n22300.txt'
    elif data_set_number==4: 
        file_path = 'fnl4461-n4460.txt'
    elif data_set_number==5:  
        file_path = 'fnl4461-n44600.txt'
    elif data_set_number==6: 
        file_path = 'pla33810-n169045.txt'
    elif data_set_number==7: 
        file_path = 'pla33810-n33809.txt'
    elif data_set_number==8: 
         file_path = 'pla33810-n338090.txt'

    with open(file_path, 'r') as file:

        file_content = file.readlines()

    problem_info = {}
    node_coord_section = []
    items_section=[]

    for line in file_content:
        line = line.strip()
        if line.startswith('PROBLEM NAME:'):
            problem_info['PROBLEM_NAME'] = line.split(':')[-1].strip()
            
        elif line.startswith('KNAPSACK DATA TYPE:'):
            problem_info['KNAPSACK_DATA_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('DIMENSION:'):
            problem_info['DIMENSION'] = int(line.split(':')[-1].strip())
           
        elif line.startswith('NUMBER OF ITEMS:'):
            problem_info['NUMBER_OF_ITEMS'] = int(line.split(':')[-1].strip())
            
        elif line.startswith('CAPACITY OF KNAPSACK:'):
            problem_info['CAPACITY_OF_KNAPSACK'] = int(line.split(':')[-1].strip())

        elif line.startswith('MIN SPEED:'):
            problem_info['MIN_SPEED'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('MAX SPEED:'):
            problem_info['MAX_SPEED'] = float(line.split(':')[-1].strip())
          
        elif line.startswith('RENTING RATIO:'):
            problem_info['RENTING_RATIO'] = float(line.split(':')[-1].strip())
            
        elif line.startswith('EDGE_WEIGHT_TYPE:'):
            problem_info['EDGE_WEIGHT_TYPE'] = line.split(':')[-1].strip()
            
        elif line.startswith('NODE_COORD_SECTION'):
            break

    for line in file_content[len(problem_info) + 1:]:

        if line.startswith('ITEMS SECTION'):
            break

        if line.strip():
            index, x, y = map(int, line.split())
            node_coord_section.append((index, x, y))


    for line in file_content[len(node_coord_section)+len(problem_info) +2 :]:
        if line.strip():
            index, x, y, z = map(int, line.split())
            items_section.append((index, x, y,z))
    
#     print(problem_info)#--------test
#     print(node_coord_section)#--------test
#     print(items_section)#--------test
    
    dimension = problem_info['DIMENSION']
    number_of_items= problem_info['NUMBER_OF_ITEMS']
    capacity_of_knapsack = problem_info['CAPACITY_OF_KNAPSACK']
    min_speed = problem_info['MIN_SPEED']
    max_speed = problem_info['MAX_SPEED']
    renting_ration=problem_info['RENTING_RATIO']
    
    return  dimension,\
            number_of_items,\
            capacity_of_knapsack,\
            min_speed,max_speed,\
            renting_ration,\
            node_coord_section,\
            items_section

In [280]:
#Class Item

class Item:
    
    def __init__(self, item_number, value, weight, pheromone):
        
        self.number = item_number
        self.value  = value
        self.weight = weight
        self.cost_performance= round(value/weight,4)
        self.pheromone= pheromone
        
        

In [281]:
#Class TTPsolution

class TTPsolution:
    
    def __init__(self,value,cost_time,weight,city_list,pack_plan):
        
        self.solution_value = value
        self.solution_cost_time = cost_time
        self.solution_weight = weight
        self.solution_path = city_list
        self.solution_pack_plan = pack_plan


In [282]:
#BuildDistanceMatrix 
##return matrix[i][j] = distance--city i to city j

def BuildDistanceMatrix(city_number,node_coord_section):
    
    matrix=[]
    for i in range(city_number):        
        row = []
        for j in range(city_number):            
            if i == j:                
                row.append(0)
            else:        
                x1,y1=node_coord_section[i][1],node_coord_section[i][2]
                x2,y2=node_coord_section[j][1],node_coord_section[j][2]
                distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                distance_rounded = round(distance, 5)
                row.append(distance_rounded)

#                 print(i,x1,y1)------------test
#                 print(j,x2,y2)
#                 print(distance)
                
        matrix.append(row)
        
    return matrix


In [283]:
#BuildCityItemsDict
#return city_item_dict[1]=[1,2,3]----Dictionary city1 have 1,2,3 items
def BuildCityItemsDict(city_number,items_section):
    
    city_item_dict={}
    
    for i in range(city_number):
        
        every_item_in_city=[]
        
        for item in items_section:
            
            if item[3]==i+1:
                
                item_number=item[0]
                item_value=item[1]
                item_weight=item[2]
                
                #Initialize the pheromone
                item_pheromone = 1

                temp = Item(item_number,item_value,item_weight,item_pheromone)
                every_item_in_city.append(temp)
                
        city_item_dict[i+1] = every_item_in_city
        
    return city_item_dict         

In [284]:
#Initial_distance_Pheromone,Heuristic and update
def InitialisDistancePheromone(city_number):
    matrix=[]
    for i in range(city_number):
        row=[]
        for j in range(city_number):
            
            if i==j:
                temp=0
            else:
                temp=1
                
            row.append(temp)
            
        matrix.append(row)
    return matrix    

def InitialisDistanceHeuristic(city_number,distance_matrix):
    
    matrix=[]
    for i in range(city_number):
        row=[]
        for j in range(city_number):
            
            if i==j:
                temp=0
            else:
                if distance_matrix[i][j] != 0:
                    temp=round(1/distance_matrix[i][j],4)
                else:
                    temp=0
            row.append(temp)
            
        matrix.append(row)
    return matrix   

def UpdateDistanceHeuristic(current_distance_heuristic,city_number,current_city):
    
    for i in range(city_number):
        for j in range(city_number):
            if j==current_city:
                current_distance_heuristic[i][j]=0

In [297]:
#calculate Transition probabilities
#return next city and item_list
def TransitionProbabilities(travel_city_list,city_number,city_items_dict,current_distance_pheromone,\
                            current_distance_heuristic,alpha,beta):
    
    current_city = travel_city_list[-1]
    probability_matrix = []
    
    #Initialize the possibility matrix
    for i in range(city_number):
        row=[]
        for j in range(len(city_items_dict[i+1])):
            row.append(0)  
        probability_matrix.append(row)

    for i in range(city_number):
        
        for j in range(len(city_items_dict[i+1])):
            
            temp1 = current_distance_pheromone[current_city][i]*city_items_dict[i+1][j].pheromone
            temp2 = current_distance_heuristic[current_city][i]*city_items_dict[i+1][j].cost_performance

            probability_matrix[i][j] = temp1 ** alpha * temp2 ** beta
    
    sum_ = 0
    for i in range(city_number):
        for j in range(len(probability_matrix[i])):
            sum_ += probability_matrix[i][j]

    sum_pro = 0
    
    for i in range(city_number):
        row = []
        for j in range(len(probability_matrix[i])):
            sum_pro += round(probability_matrix[i][j] / sum_,4)
            probability_matrix[i][j]=sum_pro

    rand = random.random()
    print(probability_matrix)
    print(rand)
    for i in range(city_number):
        for j in range(len(probability_matrix[i])):
            if probability_matrix[i][j] >= rand:
                return i, j

    
    

In [298]:
#calculate CurrentSpeed
def CurrentSpeed(v_max,v_min,capacity_of_knapsack,current_weight):
    speed = None
    if current_weight<=capacity_of_knapsack:
        speed= v_max - ((current_weight/capacity_of_knapsack)*(v_max-v_min))
    else:
        speed=v_min 
    if speed :
        return speed
    else:
        print("speed calculate is wrong")    

In [299]:
#UpdateBestSolution
def UpdateBestSolution(best_solution, ant_solution):
    
    if best_solution==[]:
        
        best_solution.append(ant_solution)
    
    else:
        
        #Make sure this point is added, 0 abandon,1-in
        add_sign=1
        pop_list=[]
        
        for i in range(len(best_solution)):
            
            #The point is completely dominated
            if  ant_solution.solution_value < best_solution[i].solution_value \
                and ant_solution.solution_cost_time > best_solution[i].solution_cost_time:
                
                add_sign=0
                break
                
            #One past point is completely dominated
            elif  ant_solution.solution_value >= best_solution[i].solution_value \
                and ant_solution.solution_cost_time <= best_solution[i].solution_cost_time:
                pop_list.append(i)
                
        for i in reversed(pop_list):
            best_solution.pop(i)

          
        if add_sign==1:
            
            best_solution.append(ant_solution)
                
    return best_solution

In [300]:
#ant_colony
class AntColony:
    
    def __init__(self,ant_number,iteration,alpha, beta,evaporation_rate,pheromone_max,pheromone_min,city_number,\
               number_of_items, capacity_of_knapsack, min_speed, max_speed,\
               renting_ration,distance_matrix,city_items_dict):
        
        self.ant_number = ant_number
        self.iteration = iteration
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.city_number = city_number
        self.number_of_items = number_of_items
        self.capacity_of_knapsack = capacity_of_knapsack
        self.min_speed = min_speed
        self.max_speed = max_speed
        self.renting_ration=renting_ration
        self.distance_matrix = distance_matrix
        self.city_items_dict = city_items_dict  
         
    def run(self):
        
        best_solution = []

        #create pheromone
        current_distance_pheromone = InitialisDistancePheromone(self.city_number)
        current_distance_heuristic = InitialisDistanceHeuristic(self.city_number,distance_matrix)

        for iterate in range(self.iteration):  

            for ant_ in range(self.ant_number):

                #----------------------one ant start------------------------
                print("the",iterate+1, "iteration",ant_+1,"ant start")
                
                travel_city_list=[]
                pack_plan=[0] * self.number_of_items
                
                current_speed = 0
                current_weight = 0
                current_value = 0
                cost_time = 0
                
                current_distance_pheromone  = InitialisDistancePheromone(self.city_number)
                current_distance_heuristic  = InitialisDistanceHeuristic(self.city_number,distance_matrix) 
                
                #random find start city
                current_city = random.randint(0, self.city_number-1)
                travel_city_list.append(current_city)
                UpdateDistanceHeuristic(current_distance_heuristic,self.city_number,current_city)
                #The first city no choose any item

                current_speed= CurrentSpeed(max_speed,min_speed,self.capacity_of_knapsack,current_weight)
                for i in range(self.city_number-1):
                #-----------------------ant go to cities-------------------------
                    next_city,next_items= TransitionProbabilities(travel_city_list,self.city_number,self.city_items_dict,current_distance_pheromone,\
                                                                current_distance_heuristic,self.alpha,self.beta)

                    
                    if next_items!=0:
                        
                        current_value+=self.city_items_dict[next_city][next_items-1].value
                        current_weight+=self.city_items_dict[next_city][next_items-1].weight
                        current_speed = CurrentSpeed(self.max_speed,self.min_speed,self.capacity_of_knapsack,current_weight)
                        number=self.city_items_dict[next_city][next_items-1].number
                        pack_plan[number-1]=1
                    cost_time=self.distance_matrix[current_city][next_city]/current_speed
                    current_city=next_city
                    
                print("the",iterate+1, "iteration",ant+1,"ant finish")
                #-------------------------one ant finish-----------------------------
                #-----create TTP solution-----
                ant_solution=TTPsolution(current_value,cost_time,current_weight,travel_city_list,travel_package)
                the_generation_all_solution.append(ant_solution)
                best_solution=UpdateBestSolution(best_solution,ant_solution)

            #-------------------------------every ants finished------------------------     
                #print(all_travel_city_list)
                #update pheromone
           
            #print("generation",evolve,"---------------")
            #UpdatePheromone(evaporation_rate,current_pheromone_matrix,self.city_number,self.ant_number,the_generation_all_solution,distance_matrix)

        return best_solution
        

In [301]:
#show data
def ShowData(best_solution):
    
    for i in range(len(best_solution)):
        temp_1 = best_solution[i].solution_value
        temp_2 = best_solution[i].solution_cost_time
        color = plt.cm.viridis(i / len(best_solution))  
        plt.scatter(temp_1, temp_2, color=color)
    
    plt.title('Scatter Plot of Solution Value vs Solution Cost Time')
    plt.xlabel('Solution Value')
    plt.ylabel('Solution Cost Time')

    plt.show()    

In [302]:
#mian function
import math
import random
import copy
import matplotlib.pyplot as plt
#----------------------------------------Extract data from a file-----------------------------

# print(f"""
#     a280-n1395:0
#     a280-n279:1
#     a280-n2790:2
#     fnl4461-n22300:3
#     fnl4461-n4460:4
#     fnl4461-n44600:5
#     pla33810-n169045:6
#     pla33810-n33809:7
#     pla33810-n338090:8
# """)
#the data set number
#data_set_number = input("Please enter the data set number:")
#ReadFile(data_set_number)

txt_data= ReadFile(0) #----------test

city_number, number_of_items, capacity_of_knapsack, min_speed, max_speed, renting_ration, node_coord_section,\
items_section = txt_data

distance_matrix = BuildDistanceMatrix(city_number,node_coord_section)

city_items_dict = BuildCityItemsDict(city_number,items_section)

#----------------------------------------Start ant colony-----------------------------

ant_number=10
iteration=1000
evaporation_rate=0.5
alpha=1
beta=2
pheromone_max=0.5
pheromone_min=0

ant_colony = AntColony(ant_number,iteration,alpha,beta,evaporation_rate,pheromone_max,\
                       pheromone_min,city_number,number_of_items, capacity_of_knapsack, min_speed, max_speed,\
                       renting_ration,distance_matrix,city_items_dict)
        
best_solution = ant_colony.run()

#show best solution
ShowData(best_solution)


the 1 iteration 1 ant start
[[], [0.0, 0.0007, 0.0011, 0.0049, 0.0053], [0.0088, 0.0122, 0.013000000000000001, 0.013200000000000002, 0.013200000000000002], [0.013500000000000002, 0.013700000000000002, 0.015700000000000002, 0.0164, 0.0166], [0.0172, 0.0184, 0.0184, 0.0184, 0.0185], [0.0189, 0.019, 0.019, 0.0192, 0.0192], [0.019799999999999998, 0.019899999999999998, 0.020499999999999997, 0.020499999999999997, 0.021099999999999997], [0.021099999999999997, 0.0214, 0.021599999999999998, 0.021699999999999997, 0.021799999999999996], [0.021999999999999995, 0.022199999999999994, 0.022199999999999994, 0.022499999999999996, 0.022499999999999996], [0.022899999999999997, 0.022999999999999996, 0.023399999999999997, 0.023499999999999997, 0.023799999999999998], [0.023899999999999998, 0.024099999999999996, 0.024199999999999996, 0.024199999999999996, 0.024199999999999996], [0.024399999999999995, 0.024399999999999995, 0.024699999999999996, 0.024699999999999996, 0.025099999999999997], [0.02509999999999999

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: cannot unpack non-iterable NoneType object